# Multi-Symbol Wheel Strategy Backtest

This notebook processes multiple symbols and entry dates for the wheel strategy backtest.

**Features:**
- **Multi-symbol support**: Run backtest across multiple tickers (e.g., TSLA, AAPL)
- Caches API responses to minimize requests
- Tracks active positions per symbol
- Prevents same-day duplicate entries per symbol
- Per-symbol and aggregate performance metrics

**Validation:**
Start with `tickers = ['TSLA']` and `entry_dates = ['2023-06-06']` to compare with single-day notebook.

## 1. Imports & Setup

In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import os
import sys

# Load environment variables
env_path = Path("/Users/samuelminer/Projects/nissan_options/wheel_strategy/.env")
load_dotenv(env_path, override=True)
assert os.getenv("DATABENTO_API_KEY"), "DATABENTO_API_KEY not found"

import numpy as np
import pandas as pd
import databento as db
import pandas_market_calendars as mcal
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks.analytical import delta

# Initialize clients
client = db.Historical()
nyse = mcal.get_calendar("NYSE")

print("Setup complete")

## 2. Wheel Strategy Configuration

In [ ]:
CONFIG = {
    # Tickers (multiple symbols supported)
    'tickers': ['TSLA', 'AAPL'],

    # Entry dates
    # 'entry_dates': ['2023-06-06'],  # Start with one date to validate

    # OR use date range (will be converted to trading days automatically)
    'start_date': '2023-06-01',
    'end_date': '2023-06-30',

    # Timezone
    'timezone': 'America/New_York',

    # Cache directory
    'cache_dir': '../cache/',

    # Technical filter settings
    'technical_filter_enabled': False,  # Set to False to disable
    'bb_window': 20,                   # Bollinger Band lookback period
    'bb_std': 2.0,                     # Bollinger Band standard deviations
    'require_sma_entry': False,         # Entry when close <= SMA20
    'require_bb_entry': False,         # Entry when close <= lower BB (more restrictive)

    # Option filters
    'min_dte': 30,
    'max_dte': 45,
    'min_delta': 0.25,
    'max_delta': 0.35,
    'option_type': 'P',

    # Exit strategy
    'profit_target_pct': 0.50,
    'exit_dte': 21,

    # DTE calculation settings
    'use_trading_days_for_dte_filter': True,   # True = trading days, False = calendar days (for min_dte/max_dte filtering)
    'use_trading_days_for_exit': True,         # True = trading days, False = calendar days (for exit date calculation)

    # Risk-free rate for IV/delta calculation
    'risk_free_rate': 0.04,
}

# Create cache directory if needed
os.makedirs(CONFIG['cache_dir'], exist_ok=True)

# Convert date range to list of trading days if specified
if 'start_date' in CONFIG and 'end_date' in CONFIG:
    start = pd.Timestamp(CONFIG['start_date'])
    end = pd.Timestamp(CONFIG['end_date'])

    # Get trading days from NYSE calendar
    trading_days = nyse.valid_days(start_date=start, end_date=end)
    CONFIG['entry_dates'] = [d.strftime('%Y-%m-%d') for d in trading_days]

    print(f"Date range: {CONFIG['start_date']} to {CONFIG['end_date']}")
    print(f"Generated {len(CONFIG['entry_dates'])} trading days")
    print(f"First 5: {CONFIG['entry_dates'][:5]}")
    print(f"Last 5: {CONFIG['entry_dates'][-5:]}")
else:
    print(f"Entry dates: {CONFIG['entry_dates']}")

print(f"Tickers: {CONFIG['tickers']}")
print(f"Cache dir: {CONFIG['cache_dir']}")
print(f"Technical filter: {'ENABLED' if CONFIG['technical_filter_enabled'] else 'DISABLED'}")
if CONFIG['technical_filter_enabled']:
    print(f"  SMA entry (close <= SMA{CONFIG['bb_window']}): {CONFIG['require_sma_entry']}")
    print(f"  BB entry (close <= lower BB): {CONFIG['require_bb_entry']}")
print(f"\nDTE settings:")
print(f"  Filter DTE: {'trading days' if CONFIG['use_trading_days_for_dte_filter'] else 'calendar days'}")
print(f"  Exit DTE: {'trading days' if CONFIG['use_trading_days_for_exit'] else 'calendar days'}")
print(f"\nTotal combinations: {len(CONFIG['tickers'])} tickers x {len(CONFIG['entry_dates'])} dates = {len(CONFIG['tickers']) * len(CONFIG['entry_dates'])}")

## 3. Caching Functions

In [ ]:
def get_cache_path(name):
    """Get full path for a cache file"""
    return os.path.join(CONFIG['cache_dir'], f"{name}.parquet")

def load_from_cache(name):
    """Load DataFrame from cache if it exists"""
    path = get_cache_path(name)
    if os.path.exists(path):
        print(f"  [CACHE HIT] Loading {name}")
        return pd.read_parquet(path)
    return None

def save_to_cache(df, name):
    """Save DataFrame to cache"""
    path = get_cache_path(name)
    df.to_parquet(path)
    print(f"  [CACHE SAVE] Saved {name}")

print("Caching functions defined")

## 4. Helper Functions

In [ ]:
def parse_option_symbols(df):
    """Parse OPRA symbols into components"""
    sym = df["symbol"]
    
    # Split ROOT and OPRA code
    root_and_code = sym.str.split(expand=True)
    df["root"] = root_and_code[0]
    code = root_and_code[1]
    
    # Expiration: YYMMDD
    df["expiration"] = pd.to_datetime(code.str[:6], format="%y%m%d")
    
    # Call/Put flag
    df["call_put"] = code.str[6]
    
    # Strike: in 1/1000 dollars
    strike_int = code.str[7:].astype("int32")
    df["strike"] = strike_int / 1000.0
    
    return df


def add_trading_dte(df, tz="America/New_York", use_trading_days=True):
    """
    Add days-to-expiration using NYSE calendar or calendar days.
    
    Args:
        df: DataFrame with ts_event and expiration columns
        tz: Timezone for event dates
        use_trading_days: If True, count trading days. If False, count calendar days.
    """
    out = df.copy()
    
    # Event dates from ts_event column
    event_dt = pd.to_datetime(out["ts_event"]).dt.tz_convert(tz).dt.normalize()
    event_days = pd.to_datetime(event_dt.dt.date)  # tz-naive
    
    # Expiration dates
    exp_dt = pd.to_datetime(out["expiration"])
    exp_days = pd.to_datetime(exp_dt.dt.date)  # tz-naive
    
    if use_trading_days:
        # Build trading calendar
        start_date = event_days.min().date()
        end_date = exp_days.max().date()
        
        schedule = nyse.valid_days(start_date=start_date, end_date=end_date)
        schedule = pd.to_datetime(schedule).normalize().tz_localize(None)
        
        cal_index = pd.Series(np.arange(len(schedule), dtype=np.int32), index=schedule)
        
        event_idx = cal_index.reindex(event_days).to_numpy()
        exp_idx = cal_index.reindex(exp_days).to_numpy()
        
        out["dte"] = (exp_idx - event_idx - 1).astype(np.int16)
    else:
        # Calendar days (simple subtraction)
        out["dte"] = (exp_days - event_days).dt.days.astype(np.int16)
    
    return out


def calculate_exit_dte_dates(expirations, exit_dte, use_trading_days=True):
    """
    Calculate dates at exit_dte days before expiration.
    
    Args:
        expirations: Series of expiration dates
        exit_dte: Number of days before expiration to exit (0 = expiration day)
        use_trading_days: If True, count trading days. If False, count calendar days
                          (will adjust to prior market day if lands on non-trading day)
    
    Returns:
        Series of exit dates (always valid trading days)
    """
    if exit_dte == 0:
        # Exit at expiration - return expiration dates (already trading days for options)
        return expirations.apply(lambda x: pd.Timestamp(x).normalize())
    
    min_exp = expirations.min()
    max_exp = expirations.max()
    
    start_date = min_exp - pd.Timedelta(days=60)
    end_date = max_exp
    
    schedule = nyse.schedule(start_date=start_date, end_date=end_date)
    trading_days = schedule.index.tz_localize(None)
    trading_days_set = set(trading_days)
    
    results = []
    for exp in expirations:
        exp_dt = pd.Timestamp(exp).normalize()
        
        if use_trading_days:
            # Count back trading days
            valid_days = trading_days[trading_days <= exp_dt]
            
            if len(valid_days) >= exit_dte:
                target_date = valid_days[-(exit_dte)]
            else:
                target_date = valid_days[0] if len(valid_days) > 0 else exp_dt
        else:
            # Count back calendar days
            target_date = exp_dt - pd.Timedelta(days=exit_dte)
            
            # If target_date is not a trading day, find the prior trading day
            if target_date not in trading_days_set:
                prior_days = trading_days[trading_days < target_date]
                if len(prior_days) > 0:
                    target_date = prior_days[-1]
                else:
                    # Fallback to expiration if no prior trading days
                    target_date = exp_dt
        
        results.append(target_date)
    
    return pd.Series(results, index=expirations.index)


def compute_iv(row, r):
    """Compute implied volatility"""
    price = row["mid"]
    S = row["underlying_last"]
    K = row["strike"]
    t = row["dte"] / 365.0
    flag = "p" if row["call_put"] == "P" else "c"

    if not (np.isfinite(price) and np.isfinite(S) and np.isfinite(K) and t > 0):
        return np.nan
    if price <= 0 or S <= 0 or K <= 0:
        return np.nan

    try:
        return implied_volatility(price, S, K, t, r, flag)
    except Exception:
        return np.nan


def compute_delta(row, r):
    """Compute delta using IV"""
    sigma = row["iv"]
    if not np.isfinite(sigma):
        return np.nan

    S = row["underlying_last"]
    K = row["strike"]
    t = row["dte"] / 365.0
    flag = "p" if row["call_put"] == "P" else "c"

    return delta(flag, S, K, t, r, sigma)


print("Helper functions defined")

## 5. Data Fetch Functions

In [ ]:
def fetch_options_snapshot(ticker, date):
    """Fetch option chain at 15:45 ET, with caching"""
    cache_name = f"options_{ticker}_{date}"
    
    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached
    
    # Fetch from API
    print(f"  [API] Fetching options for {ticker} on {date}...")
    
    tz = CONFIG['timezone']
    start_time = pd.Timestamp(f"{date} 15:45", tz=tz)
    end_time = start_time + pd.Timedelta(minutes=1)
    
    data = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='cmbp-1',
        symbols=f'{ticker}.OPT',
        stype_in='parent',
        start=start_time,
        end=end_time,
    )
    
    df = data.to_df(tz=tz).sort_values("ts_event")
    print(f"  [API] Fetched {len(df)} records")
    
    # Save to cache
    save_to_cache(df, cache_name)
    
    return df


def fetch_equity_price(ticker, date):
    """Fetch underlying price at 15:45 ET, with caching"""
    cache_name = f"equity_{ticker}_{date}"
    
    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached['close'].iloc[0]
    
    # Fetch from API
    print(f"  [API] Fetching equity price for {ticker} on {date}...")
    
    tz = CONFIG['timezone']
    start_time = pd.Timestamp(f"{date} 15:45", tz=tz)
    end_time = start_time + pd.Timedelta(minutes=1)
    
    data = client.timeseries.get_range(
        dataset='XNAS.ITCH',
        symbols=[ticker],
        schema='ohlcv-1m',
        start=start_time,
        end=end_time,
        stype_in='raw_symbol'
    )
    
    df = data.to_df()
    print(f"  [API] Fetched equity price: ${df['close'].iloc[0]:.2f}")
    
    # Save to cache
    save_to_cache(df, cache_name)
    
    return df['close'].iloc[0]


def fetch_option_daily_ohlcv(symbol, start_date, end_date):
    """Fetch daily OHLCV for an option symbol, with caching"""
    # Clean symbol for cache filename
    cache_name = f"daily_{symbol.replace(' ', '_')}_{start_date}_{end_date}"
    
    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached
    
    # Fetch from API
    print(f"  [API] Fetching daily OHLCV for {symbol} from {start_date} to {end_date}...")
    
    data = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='ohlcv-1d',
        symbols=symbol,
        stype_in='raw_symbol',
        start=start_date,
        end=end_date,
    )
    
    df = data.to_df(tz=CONFIG['timezone'])
    print(f"  [API] Fetched {len(df)} daily records")
    
    # Save to cache
    save_to_cache(df, cache_name)
    
    return df


def fetch_option_1545_price(symbol, date):
    """Fetch option price at 15:45 ET for a specific date, with caching"""
    # Clean symbol for cache filename
    cache_name = f"option_1545_{symbol.replace(' ', '_')}_{date}"
    
    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached['close'].iloc[0]
    
    # Fetch from API
    print(f"  [API] Fetching 15:45 price for {symbol} on {date}...")
    
    exit_time = pd.Timestamp(date).tz_localize(CONFIG['timezone']).replace(hour=15, minute=45)
    
    data = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='ohlcv-1m',
        symbols=symbol,
        stype_in='raw_symbol',
        start=exit_time,
        end=exit_time + pd.Timedelta(minutes=1),
    )
    
    df = data.to_df(tz=CONFIG['timezone'])
    
    if len(df) > 0:
        exit_price = df.iloc[0]['close']
        print(f"  [API] Fetched price: ${exit_price:.2f}")
        
        # Save to cache
        save_to_cache(df, cache_name)
        
        return exit_price
    else:
        print(f"  [API] No data available")
        return None


print("Data fetch functions defined")

## 5b. Technical Filter Functions

In [ ]:
def fetch_equity_history(ticker, end_date, lookback_days=60):
    """Fetch daily equity OHLCV data for technical analysis, with caching"""
    # Calculate start date with buffer for lookback
    end_dt = pd.Timestamp(end_date)
    start_dt = end_dt - pd.Timedelta(days=lookback_days)

    cache_name = f"equity_daily_{ticker}_{start_dt.date()}_{end_dt.date()}"

    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached

    # Fetch from API
    print(f"  [API] Fetching equity history for {ticker} from {start_dt.date()} to {end_dt.date()}...")

    data = client.timeseries.get_range(
        dataset='XNAS.ITCH',
        symbols=[ticker],
        schema='ohlcv-1d',
        start=start_dt,
        end=end_dt + pd.Timedelta(days=1),
        stype_in='raw_symbol'
    )

    df = data.to_df(tz=CONFIG['timezone'])
    print(f"  [API] Fetched {len(df)} daily records")

    # Save to cache
    save_to_cache(df, cache_name)

    return df


def calculate_bollinger_bands(df, window=20, k=2.0):
    """Calculate Bollinger Bands and SMA on equity data"""
    df_bb = df.copy().sort_index()

    # Rolling stats on close
    roll = df_bb["close"].rolling(window=window, min_periods=window)
    df_bb["sma"] = roll.mean()
    df_bb["std"] = roll.std(ddof=0)

    # Bollinger Bands
    df_bb["bb_upper"] = df_bb["sma"] + k * df_bb["std"]
    df_bb["bb_lower"] = df_bb["sma"] - k * df_bb["std"]

    # Bollinger %B (position within bands)
    df_bb["bb_pctb"] = (df_bb["close"] - df_bb["bb_lower"]) / (df_bb["bb_upper"] - df_bb["bb_lower"])

    return df_bb


def check_technical_entry(ticker, entry_date, config):
    """
    Check if the technical entry conditions are met for a given date.

    Returns: (passes_filter: bool, details: dict)
    """
    if not config.get('technical_filter_enabled', False):
        return True, {'filter_enabled': False}

    # Need extra lookback for BB calculation
    lookback_days = config.get('bb_window', 20) + 40

    # Fetch equity history
    df_equity = fetch_equity_history(ticker, entry_date, lookback_days)

    # Calculate Bollinger Bands
    window = config.get('bb_window', 20)
    k = config.get('bb_std', 2.0)
    df_bb = calculate_bollinger_bands(df_equity, window=window, k=k)

    # Get the entry date row
    entry_dt = pd.Timestamp(entry_date).tz_localize(CONFIG['timezone']).normalize()

    # Find the closest date (in case entry_date is exact match or close)
    df_bb_dates = df_bb.index.normalize()

    # Try to find an exact or near match
    mask = df_bb_dates <= entry_dt
    if not mask.any():
        print(f"  [TECH FILTER] No data found for {entry_date}")
        return False, {'error': 'no_data'}

    # Get the most recent row on or before entry_date
    closest_idx = df_bb[mask].index[-1]
    row = df_bb.loc[closest_idx]

    close = row['close']
    sma = row['sma']
    bb_lower = row['bb_lower']

    # Check if we have valid BB data
    if pd.isna(sma) or pd.isna(bb_lower):
        print(f"  [TECH FILTER] Insufficient data for BB calculation on {entry_date}")
        return False, {'error': 'insufficient_data'}

    # Check entry conditions
    sma_entry = close <= sma
    bb_entry = close <= bb_lower

    details = {
        'date': closest_idx,
        'close': close,
        'sma': sma,
        'bb_lower': bb_lower,
        'sma_entry': sma_entry,
        'bb_entry': bb_entry,
    }

    # Determine if we pass the filter
    require_sma = config.get('require_sma_entry', True)
    require_bb = config.get('require_bb_entry', False)

    passes = False
    if require_bb:
        passes = bb_entry
    elif require_sma:
        passes = sma_entry
    else:
        passes = sma_entry or bb_entry  # Either condition

    return passes, details


print("Technical filter functions defined")

## 6. Exit Strategy Function

In [18]:
def backtest_exit_strategy(backtest_candidates, ticker, client, config):
    """
    Backtest exit strategy for wheel options
    
    Exit conditions:
    1. Profit target: Exit when mid-price <= 50% of premium (early exit)
       - If daily range contains exit_target_price, assume we exited at that exact price
    2. Time limit: Force exit at exit_dte using 15:45 ET price
    """
    exits = []
    exit_dte = config.get('exit_dte', 21)
    
    for idx, row in backtest_candidates.iterrows():
        symbol = row['symbol']
        
        # Normalize dates
        entry_date = pd.Timestamp(row['date']).tz_localize(None)
        expiration = pd.Timestamp(row['expiration']).tz_localize(None)
        date_exit = pd.Timestamp(row['date_exit']).tz_localize(None)
        
        # Entry details
        premium = row['mid']
        profit_target_pct = CONFIG['profit_target_pct']
        exit_target_price = premium * profit_target_pct
        cost_basis = row['strike'] * 100
        
        print(f"\nProcessing {symbol}...")
        print(f"  Entry: {entry_date.date()}, Premium: ${premium:.2f}")
        print(f"  Exit target: ${exit_target_price:.2f} ({CONFIG['profit_target_pct']*100}%)")
        print(f"  Exit date ({exit_dte} DTE): {date_exit.date()}")
        
        try:
            # Fetch daily prices for monitoring (WITH CACHING)
            start_daily = entry_date + pd.Timedelta(days=1)
            end_daily = date_exit
            
            # Validate date range
            if start_daily > end_daily:
                print(f"  Warning: Invalid date range (entry+1 > exit), skipping profit target check")
                df_daily = pd.DataFrame()
            else:
                df_daily = fetch_option_daily_ohlcv(symbol, start_daily, end_daily)
            
            # Check daily for profit target
            profit_target_hit = False
            
            for check_date, daily_row in df_daily.iterrows():
                daily_low = daily_row['low']
                daily_high = daily_row['high']
                
                # Check if exit target is within daily range
                if daily_low <= exit_target_price <= daily_high:
                    exits.append({
                        'ticker': ticker,
                        'symbol': symbol,
                        'entry_date': entry_date,
                        'exit_date': check_date.tz_localize(None),
                        'expiration': expiration,
                        'cost_basis': cost_basis,
                        'premium': premium,
                        'profit_target_pct': profit_target_pct,
                        'exit_target_price': exit_target_price,
                        'exit_reason': 'profit_target',
                        'days_held': (check_date.tz_localize(None) - entry_date).days,
                        'daily_low': daily_low,
                        'daily_high': daily_high,
                    })
                    
                    print(f"  Profit target hit on {check_date.date()} @ ${exit_target_price:.2f}")
                    print(f"    (Daily range: ${daily_low:.2f} - ${daily_high:.2f})")
                    profit_target_hit = True
                    break
            
            # If profit target not hit, force exit at exit date (WITH CACHING)
            if not profit_target_hit:
                exit_price = fetch_option_1545_price(symbol, date_exit.date())
                
                if exit_price is not None:
                    exits.append({
                        'ticker': ticker,
                        'symbol': symbol,
                        'entry_date': entry_date,
                        'exit_date': date_exit,
                        'expiration': expiration,
                        'cost_basis': cost_basis,
                        'premium': premium,
                        'exit_target_price': exit_target_price,
                        'exit_price': exit_price,
                        'exit_reason': f'time_limit_{exit_dte}dte',
                        'days_held': (date_exit - entry_date).days,
                        'daily_low': None,
                        'daily_high': None,
                    })
                    
                    print(f"  Time limit exit on {date_exit.date()} @ 15:45 ET: ${exit_price:.2f}")
                else:
                    print(f"  No data for {exit_dte} DTE exit")
                    
        except Exception as e:
            print(f"  Error: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    # Create results DataFrame
    exits_df = pd.DataFrame(exits)
    
    # Calculate P&L
    if len(exits_df) > 0:
        exits_df['exit_pnl'] = exits_df['premium'] - exits_df['exit_price']
        exits_df['exit_pnl_pct'] = (exits_df['exit_pnl'] / exits_df['premium']) * 100
        exits_df['roc'] = (exits_df['exit_pnl'] / exits_df['cost_basis']) * 100
    
    return exits_df


print("Exit strategy function defined")

Exit strategy function defined


## 7. Process Entry Date Function

In [19]:
def process_entry_date(entry_date, ticker, positions_df, config):
    """
    Process a single entry date for a single ticker:
    1. Fetch options chain
    2. Parse symbols, calc DTE, IV, delta
    3. Filter candidates
    4. Remove same-day duplicates (symbols already in positions_df for this date/ticker)
    5. Run exit strategy
    
    Returns: (new_positions_df, exits_df)
    """
    print(f"\n{'='*60}")
    print(f"Processing {ticker} on {entry_date}")
    print('='*60)
    
    r = config['risk_free_rate']
    use_trading_days_filter = config.get('use_trading_days_for_dte_filter', True)
    use_trading_days_exit = config.get('use_trading_days_for_exit', True)
    exit_dte = config.get('exit_dte', 21)
    
    # 1. Fetch options chain
    df_opts = fetch_options_snapshot(ticker, entry_date)
    
    # 2. Parse symbols
    df_opts = parse_option_symbols(df_opts)
    
    # 3. Add DTE (using config setting for trading vs calendar days)
    df_opts = add_trading_dte(df_opts, use_trading_days=use_trading_days_filter)
    
    # 4. Fetch underlying price
    underlying_price = fetch_equity_price(ticker, entry_date)
    
    # 5. Keep only rows with quotes
    quotes = df_opts[df_opts["bid_px_00"].notna() & df_opts["ask_px_00"].notna()].copy()
    quotes["mid"] = (quotes["bid_px_00"] + quotes["ask_px_00"]) / 2
    
    # 6. Collapse to one row per contract (latest quote)
    chain_snapshot = (
        quotes
        .sort_values("ts_event")
        .groupby(["symbol", "expiration", "strike", "call_put"])
        .tail(1)
        .copy()
    )
    chain_snapshot["underlying_last"] = underlying_price
    
    # 7. Calculate IV and delta
    chain_snapshot["iv"] = chain_snapshot.apply(lambda row: compute_iv(row, r), axis=1)
    chain_snapshot["delta"] = chain_snapshot.apply(lambda row: compute_delta(row, r), axis=1)
    
    # 8. Calculate exit dates (using config setting for trading vs calendar days)
    chain_snapshot['date_exit'] = calculate_exit_dte_dates(
        chain_snapshot['expiration'], 
        exit_dte,
        use_trading_days=use_trading_days_exit
    )
    
    # 9. Add entry date
    chain_snapshot['date'] = chain_snapshot['ts_event'].dt.date
    
    # 10. Filter candidates
    candidates = chain_snapshot[
        (chain_snapshot["call_put"] == config['option_type'])
        & chain_snapshot["dte"].between(config['min_dte'], config['max_dte'])
        & chain_snapshot["delta"].abs().between(config['min_delta'], config['max_delta'])
    ].copy()
    
    print(f"\nFound {len(candidates)} candidates passing filters")
    print(f"  DTE calculation: {'trading days' if use_trading_days_filter else 'calendar days'}")
    print(f"  Exit date calculation: {'trading days' if use_trading_days_exit else 'calendar days'}")
    
    if len(candidates) == 0:
        return pd.DataFrame(), pd.DataFrame()
    
    # 11. Filter out same-day duplicates (symbols already held from this entry date for this ticker)
    if len(positions_df) > 0:
        # Only filter if we have positions from the SAME entry date AND ticker
        same_date_positions = positions_df[
            (positions_df['entry_date'] == entry_date) & 
            (positions_df['ticker'] == ticker)
        ]
        if len(same_date_positions) > 0:
            held_symbols = same_date_positions['symbol'].tolist()
            candidates = candidates[~candidates['symbol'].isin(held_symbols)]
            print(f"After removing same-day duplicates: {len(candidates)} candidates")
    
    if len(candidates) == 0:
        return pd.DataFrame(), pd.DataFrame()
    
    # 12. Create backtest candidates
    backtest_candidates = candidates.copy()
    backtest_candidates['cost_basis'] = backtest_candidates['underlying_last'] * 100 - backtest_candidates['mid']
    backtest_candidates['premium'] = backtest_candidates['mid']
    backtest_candidates['exit_target_price'] = CONFIG['profit_target_pct'] * backtest_candidates['premium']
    backtest_candidates = backtest_candidates[[
        'symbol', 'date_exit', 'cost_basis', 'premium', 'exit_target_price',
        'date', 'dte', 'expiration', 'mid', 'strike'
    ]]
    
    print(f"\nBacktest candidates:")
    print(backtest_candidates[['symbol', 'strike', 'dte', 'premium', 'date_exit']].to_string())
    
    # 13. Run exit strategy
    exits_df = backtest_exit_strategy(backtest_candidates, ticker, client, config)
    
    # 14. Create new positions (entry info for tracking)
    new_positions = backtest_candidates.copy()
    new_positions['entry_date'] = entry_date
    new_positions['ticker'] = ticker
    
    return new_positions, exits_df


print("Process entry date function defined")

Process entry date function defined


## 8. Main Backtest Loop

In [20]:
# Initialize tracking DataFrames
positions_df = pd.DataFrame()  # Active positions
all_exits_df = pd.DataFrame()  # All completed trades
skipped_entries = []  # Ticker/date combinations that failed technical filter

# Process each ticker and entry date combination
for ticker in CONFIG['tickers']:
    print(f"\n{'#'*70}")
    print(f"# Processing ticker: {ticker}")
    print('#'*70)

    for entry_date in CONFIG['entry_dates']:

        # Check technical filter first (per ticker)
        if CONFIG.get('technical_filter_enabled', False):
            passes_filter, tech_details = check_technical_entry(
                ticker, entry_date, CONFIG
            )

            if not passes_filter:
                print(f"\n[SKIPPED] {ticker} on {entry_date} - Failed technical filter")
                if 'close' in tech_details:
                    print(f"  Close: ${tech_details['close']:.2f}, SMA: ${tech_details['sma']:.2f}, BB Lower: ${tech_details['bb_lower']:.2f}")
                skipped_entries.append({'ticker': ticker, 'date': entry_date, 'reason': 'technical_filter', **tech_details})
                continue
            else:
                print(f"\n[PASSED] {ticker} on {entry_date} - Technical filter passed")
                if 'close' in tech_details:
                    print(f"  Close: ${tech_details['close']:.2f}, SMA: ${tech_details['sma']:.2f}")
                    print(f"  SMA entry: {tech_details['sma_entry']}, BB entry: {tech_details['bb_entry']}")

        # Process this ticker/date combination
        new_positions, exits = process_entry_date(
            entry_date=entry_date,
            ticker=ticker,
            positions_df=positions_df,
            config=CONFIG
        )

        # Add new positions
        if len(new_positions) > 0:
            positions_df = pd.concat([positions_df, new_positions], ignore_index=True)

        # Accumulate exits
        if len(exits) > 0:
            all_exits_df = pd.concat([all_exits_df, exits], ignore_index=True)

print(f"\n{'='*70}")
print("BACKTEST COMPLETE")
print('='*70)
print(f"Tickers processed: {CONFIG['tickers']}")
total_combinations = len(CONFIG['tickers']) * len(CONFIG['entry_dates'])
print(f"Total ticker/date combinations: {total_combinations}")
if CONFIG.get('technical_filter_enabled', False):
    print(f"Combinations passing technical filter: {total_combinations - len(skipped_entries)}")
    print(f"Combinations skipped (failed filter): {len(skipped_entries)}")
    if len(skipped_entries) > 0:
        skipped_df = pd.DataFrame(skipped_entries)
        print(f"\nSkipped by ticker:")
        print(skipped_df['ticker'].value_counts().to_string())
print(f"\nTotal positions entered: {len(positions_df)}")
print(f"Total exits: {len(all_exits_df)}")



######################################################################
# Processing ticker: TSLA
######################################################################

Processing TSLA on 2023-06-01
  [CACHE HIT] Loading options_TSLA_2023-06-01
  [CACHE HIT] Loading equity_TSLA_2023-06-01

Found 2 candidates passing filters
  DTE calculation: trading days
  Exit date calculation: trading days

Backtest candidates:
                                                    symbol  strike  dte  premium  date_exit
ts_recv                                                                                    
2023-06-01 15:45:59.509547226-04:00  TSLA  230721P00195000   195.0   33    10.50 2023-06-22
2023-06-01 15:45:59.669283153-04:00  TSLA  230721P00190000   190.0   33     8.75 2023-06-22

Processing TSLA  230721P00195000...
  Entry: 2023-06-01, Premium: $10.50
  Exit target: $5.25 (50.0%)
  Exit date (21 DTE): 2023-06-22
  [CACHE HIT] Loading daily_TSLA__230721P00195000_2023-06-02 00:00:00_2023-06

KeyError: 'exit_price'

## 9. Results - Aggregate

In [ ]:
if len(all_exits_df) > 0:
    print("\n" + "="*60)
    print("AGGREGATE RESULTS (All Tickers)")
    print("="*60)
    
    print("\nExit reasons:")
    print(all_exits_df['exit_reason'].value_counts())
    
    print("\nP&L Summary:")
    print(all_exits_df[['exit_pnl', 'exit_pnl_pct', 'roc']].describe())
    
    print(f"\nTotal P&L: ${all_exits_df['exit_pnl'].sum():.2f}")
    print(f"Average ROC: {all_exits_df['roc'].mean():.2f}%")
    print(f"Win Rate: {(all_exits_df['exit_pnl'] > 0).mean() * 100:.1f}%")
    
    print("\nAll exits:")
    display_cols = ['ticker', 'symbol', 'entry_date', 'exit_date', 'premium', 'exit_price', 
                   'exit_pnl', 'roc', 'exit_reason']
    print(all_exits_df[display_cols].to_string())
else:
    print("No exits recorded")

## 10. Results - By Ticker

In [ ]:
if len(all_exits_df) > 0:
    print("\n" + "="*60)
    print("RESULTS BY TICKER")
    print("="*60)
    
    for ticker in CONFIG['tickers']:
        ticker_exits = all_exits_df[all_exits_df['ticker'] == ticker]
        
        if len(ticker_exits) == 0:
            print(f"\n{ticker}: No exits")
            continue
            
        print(f"\n{'-'*40}")
        print(f"{ticker}")
        print(f"{'-'*40}")
        
        print(f"  Trades: {len(ticker_exits)}")
        print(f"  Total P&L: ${ticker_exits['exit_pnl'].sum():.2f}")
        print(f"  Avg P&L: ${ticker_exits['exit_pnl'].mean():.2f}")
        print(f"  Avg ROC: {ticker_exits['roc'].mean():.2f}%")
        print(f"  Win Rate: {(ticker_exits['exit_pnl'] > 0).mean() * 100:.1f}%")
        print(f"  Avg Days Held: {ticker_exits['days_held'].mean():.1f}")
        
        print(f"\n  Exit reasons:")
        for reason, count in ticker_exits['exit_reason'].value_counts().items():
            print(f"    {reason}: {count}")
else:
    print("No exits recorded")

## 11. Ticker Comparison Summary

In [ ]:
if len(all_exits_df) > 0:
    print("\n" + "="*60)
    print("TICKER COMPARISON")
    print("="*60)
    
    comparison = all_exits_df.groupby('ticker').agg({
        'exit_pnl': ['count', 'sum', 'mean'],
        'roc': 'mean',
        'days_held': 'mean',
    }).round(2)
    
    comparison.columns = ['Trades', 'Total P&L', 'Avg P&L', 'Avg ROC %', 'Avg Days']
    
    # Add win rate
    win_rates = all_exits_df.groupby('ticker').apply(
        lambda x: (x['exit_pnl'] > 0).mean() * 100
    ).round(1)
    comparison['Win Rate %'] = win_rates
    
    print(comparison.to_string())
else:
    print("No exits recorded")

## 12. Validation

To validate, set:
- `tickers = ['TSLA']`
- `entry_dates = ['2023-06-06']`

Expected TSLA results:

| Symbol | Entry Date | Exit Date | Exit Reason | Premium | Exit Price |
|--------|------------|-----------|-------------|---------|------------|
| TSLA 230721P00200000 | 2023-06-06 | 2023-06-08 | profit_target | ~$7.85 | ~$3.92 |
| TSLA 230721P00205000 | 2023-06-06 | 2023-06-08 | profit_target | ~$9.53 | ~$4.76 |

In [ ]:
# Validation check for TSLA
expected_tsla_symbols = ['TSLA  230721P00200000', 'TSLA  230721P00205000']

if len(all_exits_df) > 0:
    print("Validation check (TSLA):")
    for sym in expected_tsla_symbols:
        match = all_exits_df[all_exits_df['symbol'] == sym]
        if len(match) > 0:
            row = match.iloc[0]
            print(f"  {sym}:")
            print(f"    Exit date: {row['exit_date']}")
            print(f"    Exit reason: {row['exit_reason']}")
            print(f"    Premium: ${row['premium']:.2f}")
            print(f"    Exit price: ${row['exit_price']:.2f}")
        else:
            print(f"  {sym}: NOT FOUND (may not be in tickers list)")